In [1]:
import requests


In [2]:
# Eurostat API connect and query generation using the Query generator from Eurostat
eurostat_url_service = 'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/'

queryEurostat = "for_area?groupedIndicators=1&filterNonGeo=1&precision=1&indic_fo=FOR&unit=THS_HA&time=1990&time=2000&time=2010&time=2020"
rest_petition = eurostat_url_service + queryEurostat
response = requests.get(rest_petition)

# Print the Query in the URL
rest_petition

'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/for_area?groupedIndicators=1&filterNonGeo=1&precision=1&indic_fo=FOR&unit=THS_HA&time=1990&time=2000&time=2010&time=2020'

In [3]:
# As we know that the answer will be in JSON format..

import json

jsonEurostat = json.loads(response.text)
# jsonEurostat
# the API response looks like that, in this case in the initial properties, we can see the "label" 
# title declaring = 'Area of wooded land (source: FAO - FE)'
# notice also the update date in order to check that the service is constantly updated.

In [4]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="App_prueba")

# As we are looking to represent the data in a map we will have to use a geocoding system to
# get and recognize the country codes so we import geopy.geocoders and pycountry_convert

# By other hand we must know that Eurostat give the data by countries in order, 
# so as we asked for 4 different year data collections we first must "group" the records
# by country and year

countries = {} #for now we would work with the data as a JSON not as an array
# lets use the JSON working format to get to the values we are searching for,
# in this case we want to acces to "dimension/geo/category/label" in order to get the
# relation between the unique country name and the country codes(primary keys)
# We select also tha country index if the primary key is the same as the country index 
# we establish the relation
for label in jsonEurostat['dimension']['geo']['category']['label']:
    for index in jsonEurostat['dimension']['geo']['category']['index']:
        if label == index:
            countries [jsonEurostat['dimension']['geo']['category']['index'][index]] = jsonEurostat['dimension']['geo']['category']['label'][label].replace(" (until 1990 former territory of the FRG)","")
            break

values= [] #We will build an Array with the country names
# We start defining our output table, you can notice that we insert the geolocalitation and
# the geocoding values
def camposTabla(): 
    valor['Pais'] = countries[countryId]
    valor['Valor'] = jsonEurostat['value'][dataId]
#     valor['numYear'] = x
    valor['Year']= year
    valor['Code']= country_name_to_country_alpha2(countries[countryId], cn_name_format="default")
    valor['latitude']= loc.latitude
    valor['longitude']= loc.longitude
    values.append(valor)
# As we have 4 values for each country and in the JSPN the years have a ID between 0 and 3 
# 0:1990 , 1:2000, 2: 2010, 3:2020 we would need to group the values
year= 1990
x = 0
for dataId in jsonEurostat['value']:
    groupId = (int(int(dataId)/4))
    for countryId in countries:
        if str(groupId) == str(countryId):
            valor = {}
            loc =geolocator.geocode(countries[countryId])
            if x == 0:
                camposTabla()
                x+=1
                year = 2000
                break
            if x == 1:
                camposTabla()
                x+=1
                year = 2010
                break
            if x == 2:
                camposTabla()
                x+=1
                year = 2020
                break
            if x == 3:
                camposTabla()
                x=0
                year = 1990
                break

# values
# we can see how we ordered the data


In [5]:

import pandas as pd
# pd.set_option('display.max_rows', 500)
df = pd.DataFrame(values)
year_1990=df["Year"]==1990
filtro_year_1990=df[year_1990]
year_2000=df["Year"]==2000
filtro_year_2000=df[year_2000]
year_2010=df["Year"]==2010
filtro_year_2010=df[year_2010]
year_2020=df["Year"]==2020
filtro_year_2020=df[year_2020]

# We can calculate the mean "green" surface
df2 = df.groupby(["Pais","Code","latitude","longitude"]).Valor.mean()
df2 = df2.to_frame()





In [6]:
# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster
import branca.colormap as cm
#empty map
linear = cm.LinearColormap(["green", "yellow", "red"], vmin=100, vmax=100000)

world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(df)):
        lat = df.iloc[i]['latitude']
        long = df.iloc[i]['longitude']
        radius=10
        popup_text = """<h4>Country Name :</h4> {}<br>
                    <h4>Area of Wooded Land :</h4> {}<br>
                    <h4>Year:</h4> {}<br>"""
        popup_text = popup_text.format(df.iloc[i]['Pais'],
                                   df.iloc[i]['Valor'],
                                       df.iloc[i]['Year']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map


In [7]:
import geopip

geopip.search(lng=4.910248, lat=50.850981)

{'FIPS': 'BE',
 'ISO2': 'BE',
 'ISO3': 'BEL',
 'UN': 56,
 'NAME': 'Belgium',
 'AREA': 0,
 'POP2005': 10398049,
 'REGION': 150,
 'SUBREGION': 155,
 'LON': 4.664,
 'LAT': 50.643}

In [8]:
from shapely.geometry import Point

# combine lat and lon column to a shapely Point() object
df['geometry'] = df.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)

In [9]:
import geopandas
import fiona
df = geopandas.GeoDataFrame(df,geometry='geometry') 
# df.to_json(r'C:\Users\gasca\Desktop\Export_geometry.json')
# df.to_json(r'C:\Users\gasca\Desktop\Export_geometry.json')
df
# df.to_file(r'C:\Users\gasca\Desktop\Export_geometry.geojson', driver='GeoJSON')  

,Pais,Valor,Year,Code,latitude,longitude,geometry
0,Austria,3775.67,1990,AT,47.200000,13.200000,POINT (13.20000 47.20000)
1,Austria,3838.14,2000,AT,47.200000,13.200000,POINT (13.20000 47.20000)
2,Austria,3863.20,2010,AT,47.200000,13.200000,POINT (13.20000 47.20000)
3,Austria,3899.15,2020,AT,47.200000,13.200000,POINT (13.20000 47.20000)
4,Bosnia and Herzegovina,2210.00,1990,BA,44.305348,17.596147,POINT (17.59615 44.30535)
...,...,...,...,...,...,...,...
119,Slovakia,1925.90,2020,SK,48.741152,19.452865,POINT (19.45286 48.74115)
120,United Kingdom,2778.00,1990,GB,54.702354,-3.276575,POINT (-3.27658 54.70235)
121,United Kingdom,2954.00,2000,GB,54.702354,-3.276575,POINT (-3.27658 54.70235)
122,United Kingdom,3059.00,2010,GB,54.702354,-3.276575,POINT (-3.27658 54.70235)
